In [ ]:
import torch
from pyannote.database import get_protocol, FileFinder

preprocessors = {'audio': FileFinder()}
protocol = get_protocol('VOXCON.SpeakerDiarization.Sample', preprocessors=preprocessors)

pipeline = torch.hub.load('pyannote/pyannote-audio', 'dia_dihard')

In [ ]:
import time
print(f'{time.strftime("%H:%M:%S")}')

In [ ]:
import time
from pyannote.metrics.diarization import DiarizationErrorRate


ders = []
diarizations = []

metric = DiarizationErrorRate()

for file in protocol.test():
    diarization = pipeline(file)
    diarizations.append(diarization)
    
    groundtruth = file["annotation"]
    der = metric(groundtruth, diarization)
    ders.append(der)
    print(f'DER = {100 * der:.1f}% {time.strftime("%H:%M:%S")}')

In [ ]:
diarization

In [ ]:
ders

In [ ]:
groundtruth

In [ ]:
diarizations

In [ ]:
touch ./lists/voxconverse/sample.der

In [ ]:
derPath = "/home/jovyan/work/uis-rnn-voxsrc20/lists/voxconverse/sample.der"
with open(derPath, "w") as derFile:
    derFile.write("\n".join(ders))

In [ ]:
from pyannote.database.util import load_rttm
rttm = load_rttm('/home/jovyan/work/uis-rnn-voxsrc20/lists/voxconverse/sample.rttm')[test_file['uri']]
test_file['uri']

In [ ]:
from pyannote.audio.features import RawAudio
from pyannote.core import Segment, notebook
from IPython.display import Audio

# make notebook visualization zoom on 600s < t < 660s time range
# EXCERPT = Segment(600, 660)
# notebook.crop = EXCERPT

# visualize excerpt groundtruth
groundtruth

file = {'uri': test_file['uri'], 'audio': test_file['audio']}

# load audio waveform, crop excerpt, and play it
waveform = RawAudio(sample_rate=16000)(file).data
Audio(data=waveform.squeeze(), rate=16000, autoplay=True)

In [ ]:
for turn, _, speaker in diarization.itertracks(yield_label=True):
    print(f'Speaker "{speaker}" speaks between t={turn.start:.1f}s and t={turn.end:.1f}s.')

mapping = metric.optimal_mapping(groundtruth, diarization)
diarization.rename_labels(mapping=mapping)